# 0. Import, data read, function definition

In [130]:
import os, sys
import copy
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge

In [4]:
!ls data

sample_submission.csv  test.csv  train.csv


In [5]:
test = pd.read_csv("data/test.csv")
train = pd.read_csv("data/train.csv")

print(train.columns)

Index(['id', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7',
       'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14',
       'target'],
      dtype='object')


In [10]:
X_test = test.iloc[:,1:]
X_train = train.iloc[:,1:-1]
Y_train = train.iloc[:,-1]

print(X_test.shape, X_train.shape, Y_train.shape) 
# our input space has 14 dimensions
# with 200 000 test examples, and 300 000 train examples
# output space has 1 dimension

(200000, 14) (300000, 14) (300000,)


In [156]:
def kernelize(X,kernel="linear"):
    """
    inputs:
        apply linear kernel to our data
        X : pandas dataframe, containing dimensions in columns
    returns :
        X_new : dataframe resulting of the application of the kernel on X dataframe
    """
    new_X = []
    X_columns = X.columns
    N_columns = len(X.columns)
    new_columns = []
    if kernel =="linear":
        for i in range(N_columns):
            for j in range(N_columns):
                if i <= j : # since our kernel is symetric, we need can compute only half of the operations
                    col0, col1 = X_columns[i], X_columns[j]
                    new_columns.append("k(" + col0 + "," + col1 + ")")
                    new_X.append(copy.deepcopy(np.multiply(X[col0], X[col1])))
    new_X = pd.concat(new_X, axis=1)
    new_X.columns = new_columns
    return new_X

# 1. MKL

## 1.1. Kernel method with a chosen kernel

In [157]:
# We apply product_kernel

X_test_kernel = kernelize(X_test)
X_train_kernel = kernelize(X_train)

print(X_train_kernel.shape, X_test_kernel.shape) # we get a 104 dimension space

(300000, 105) (200000, 105)


## 1.2. Ridge regression in the feature space

In [158]:
regressor = Ridge()

In [159]:
regressor.fit(X_train, Y_train)

Ridge()

In [160]:
Y_predict = regressor.predict(X_test)

In [161]:
print("predicted variance :", Y_predict.var(),"\ntraining variance :", Y_train.var())
print("\npredicted mean :", Y_predict.mean(),"\ntraining mean :", Y_train.mean())
print("\npredicted min :", Y_predict.min(),"\ntraining min :", Y_train.min())
print("\npredicted max :", Y_predict.max(),"\ntraining max :", Y_train.max())

predicted variance : 0.009986963389501852 
training variance : 0.5373928423339634

predicted mean : 7.905556778801167 
training mean : 7.905661328316826

predicted min : 7.369461306345791 
training min : 0.0

predicted max : 8.558680598787495 
training max : 10.267568500800396


In [151]:
# The classifier predict values that are in a shorter interval comparing to what we get in the training set
# It would be interesting to search why
# It may be a problem if test and train sets are constructed by random sampling per example